# Análise de Atrasos de Voos: Insights de 2020 a 2022

Este notebook compila a análise de dados de atrasos de voos e inclui uma reflexão final para o CTO da empresa, com base nos insights gerados.

## Introdução

Este artigo apresenta uma análise de dados relacionados a atrasos de voos nos anos de 2020, 2021 e 2022. Os dados foram extraídos de 36 arquivos CSV, totalizando mais de 2,4 milhões de linhas, e focam em métricas como o número total de voos, taxas de atrasos, aeroportos e companhias aéreas mais afetadas, além de tendências temporais. Atrasos foram definidos como voos com mais de 15 minutos de demora na partida. Essa análise objetiva fornecer insights baseados em dados para profissionais do setor aéreo, pesquisadores e stakeholders interessados em eficiência operacional.

In [137]:
import pandas as pd
import numpy as np
import glob
import plotly.express as px
from IPython.display import display
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)

# ---------------------------
# CONFIGURAÇÃO DE CAMINHOS
# ---------------------------
caminho_arquivos = '/content/drive/MyDrive/Voos/crvs/*.csv'  # ajuste conforme necessário
caminho_aeroportos = '/content/drive/MyDrive/Voos/flat-ui/flat-ui__data-Sat Sep 06 2025.csv'  # ajuste conforme necessário

coluna_companhia = 'ICAO Empresa Aérea'

# ---------------------------
# LEITURA ROBUSTA DOS CSVs
# ---------------------------
lista_arquivos = glob.glob(caminho_arquivos)

if not lista_arquivos:
    raise FileNotFoundError(
        f"Nenhum arquivo encontrado em '{caminho_arquivos}'. "
        "Verifique o caminho e os arquivos CSV."
    )

print(f"{len(lista_arquivos)} arquivo(s) encontrado(s). Exemplos: {lista_arquivos[:5]}")

dfs = []
for arquivo in lista_arquivos:
    lido = False
    last_error = None
    for skip in (1, 0):
        for sep in (';', ','):
            try:
                df_temp = pd.read_csv(arquivo, sep=sep, encoding='utf-8-sig', skiprows=skip)
                if df_temp is not None and not df_temp.empty:
                    dfs.append(df_temp)
                    lido = True
                    break
            except Exception as e:
                last_error = e
        if lido:
            break
    if not lido:
        print(f"Aviso: não foi possível ler '{arquivo}' (último erro: {last_error}).")

if not dfs:
    raise ValueError(
        "Nenhum arquivo CSV válido foi lido. Possíveis causas:\n"
        "- Arquivos não existem no caminho informado;\n"
        "- Arquivos usam separador diferente de ';' ou ',';\n"
        "- Arquivos têm estrutura/cabeçalho que requer ajuste de skiprows.\n"
        "Verifique os arquivos e ajuste 'caminho_arquivos' ou parâmetros de leitura."
    )

df_voos = pd.concat(dfs, ignore_index=True)
print(f"Concatenação realizada: {len(dfs)} arquivo(s) combinados -> {len(df_voos)} linhas no DataFrame resultante.")

# ---------------------------
# CHECAGEM DE COLUNAS ESSENCIAIS
# ---------------------------
colunas_necessarias = [
    'Situação Voo', 'Partida Prevista', 'Partida Real',
    'Chegada Prevista', 'Chegada Real', 'ICAO Aeródromo Origem'
]

colunas_faltantes = [c for c in colunas_necessarias if c not in df_voos.columns]
if colunas_faltantes:
    raise KeyError(
        "Colunas necessárias ausentes no dataset de voos: "
        + ", ".join(colunas_faltantes)
        + ". Verifique os nomes das colunas nos CSVs."
    )



36 arquivo(s) encontrado(s). Exemplos: ['/content/drive/MyDrive/Voos/crvs/VRA_20201.csv', '/content/drive/MyDrive/Voos/crvs/VRA_20204.csv', '/content/drive/MyDrive/Voos/crvs/VRA_20205.csv', '/content/drive/MyDrive/Voos/crvs/VRA_20203.csv', '/content/drive/MyDrive/Voos/crvs/VRA_20202.csv']
Concatenação realizada: 36 arquivo(s) combinados -> 2424025 linhas no DataFrame resultante.


## Visão Geral dos Dados

A análise envolveu a concatenação de 36 arquivos CSV, resultando em um DataFrame com 2.424.025 linhas. Os anos considerados foram 2020, 2021 e 2022, com o número total de voos analisados variando de aproximadamente 320 mil a mais de 810 mil por ano. A taxa média de atrasos situou-se entre 11% e 15%. Aeroportos como o Aeroporto Internacional de Miami e o Aeroporto Humberto Delgado de Lisboa apareceram de forma recorrente como locais com maior incidência de atrasos, enquanto companhias aéreas como AZU e GLO lideraram as estatísticas de atrasos.

In [138]:
# ---------------------------
# FILTRAGEM E PREPARO
# ---------------------------
# Mantém apenas voos que foram realizados
df_realizados = df_voos[df_voos['Situação Voo'] == 'REALIZADO'].copy()
if df_realizados.empty:
    raise ValueError("Não há voos com 'Situação Voo' == 'REALIZADO' no dataset.")

# Converte colunas de data e hora para datetime (com coerção de erros)
for col in ['Partida Prevista', 'Partida Real', 'Chegada Prevista', 'Chegada Real']:
    df_realizados[col] = pd.to_datetime(df_realizados[col], errors='coerce')

# Remove registros inválidos sem datas completas
df_realizados = df_realizados.dropna(subset=['Partida Prevista', 'Partida Real', 'Chegada Prevista', 'Chegada Real'])
if df_realizados.empty:
    raise ValueError("Após conversão de datas, não há registros com datas válidas em todas as colunas de tempo.")

# Calcula atrasos em minutos (não negativos)
df_realizados['Atraso Partida (min)'] = ((df_realizados['Partida Real'] - df_realizados['Partida Prevista']).dt.total_seconds() / 60).clip(lower=0)
df_realizados['Atraso Chegada (min)'] = ((df_realizados['Chegada Real'] - df_realizados['Chegada Prevista']).dt.total_seconds() / 60).clip(lower=0)

# Marca voos como atrasados se a partida atrasou mais de 15 minutos
df_realizados['Voo Atrasado'] = df_realizados['Atraso Partida (min)'] > 15

# Derivadas: dia da semana, período do dia e ano
df_realizados['Dia Semana'] = df_realizados['Partida Prevista'].dt.day_name()
df_realizados['Periodo Dia'] = pd.cut(df_realizados['Partida Prevista'].dt.hour,
                                      bins=[0,6,12,18,24],
                                      labels=['Madrugada','Manhã','Tarde','Noite'],
                                      right=False)
df_realizados['Ano'] = df_realizados['Partida Prevista'].dt.year
df_realizados = df_realizados.dropna(subset=['Ano'])

In [139]:
# ---------------------------
# FUNÇÃO: AJUSTE LINEAR ROBUSTO
# ---------------------------
def robust_linear_fit(x, y):
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    mask = np.isfinite(x) & np.isfinite(y)
    x, y = x[mask], y[mask]
    if x.size < 2:
        return 0.0, float(np.nan)
    if np.allclose(x, x[0]):
        return 0.0, float(y.mean())
    A = np.vstack([x, np.ones_like(x)]).T
    try:
        slope, intercept = np.linalg.lstsq(A, y, rcond=None)[0]
        slope = float(slope) if np.isfinite(slope) else 0.0
        intercept = float(intercept) if np.isfinite(intercept) else float(y.mean())
        return slope, intercept
    except Exception:
        return 0.0, float(y.mean())

# ---------------------------
# JUNÇÃO COM DADOS DE AEROPORTOS (SE DISPONÍVEL)
# ---------------------------
try:
    df_aeroportos = pd.read_csv(caminho_aeroportos)
    # Checa colunas comuns e reduz ao necessário (se existirem)
    cols_existentes = [c for c in ['ident','name','municipality','iso_country'] if c in df_aeroportos.columns]
    if 'ident' in cols_existentes and 'name' in cols_existentes:
        df_realizados = df_realizados.merge(df_aeroportos[['ident','name','municipality','iso_country']],
                                            left_on='ICAO Aeródromo Origem', right_on='ident', how='left')
    else:
        print("Arquivo de aeroportos lido, mas colunas 'ident' e/ou 'name' não encontradas. Usando código ICAO como nome do aeroporto.")
        df_realizados['name'] = df_realizados['ICAO Aeródromo Origem']
except FileNotFoundError:
    print(f"Aviso: arquivo de aeroportos '{caminho_aeroportos}' não encontrado. Usando código ICAO como nome do aeroporto.")
    df_realizados['name'] = df_realizados['ICAO Aeródromo Origem']
except Exception as e:
    print(f"Aviso ao ler arquivo de aeroportos: {e}. Usando código ICAO como nome do aeroporto.")
    df_realizados['name'] = df_realizados['ICAO Aeródromo Origem']

# Se, por acaso, 'name' ainda não existir, cria a partir do ICAO
if 'name' not in df_realizados.columns:
    df_realizados['name'] = df_realizados['ICAO Aeródromo Origem']

# ---------------------------
# LISTA DE ANOS PARA ANÁLISE (REMOVE 2023 SE APARECER)
# ---------------------------
anos_disponiveis = sorted(df_realizados['Ano'].dropna().unique())
anos_disponiveis = [ano for ano in anos_disponiveis if ano != 2023]
if not anos_disponiveis:
    raise ValueError("Nenhum ano disponível para análise após filtragem (removido 2023 se presente).")

print(f"Anos disponíveis para análise: {anos_disponiveis}")

# ---------------------------
# UTILITÁRIAS DE FORMATAÇÃO
# ---------------------------
def top_list_to_text(df, col_key, col_value, top_n=5):
    items = df.sort_values(col_value, ascending=False).head(top_n)
    return "; ".join([f"{row[col_key]} ({int(row[col_value])})" for _, row in items.iterrows()])

Anos disponíveis para análise: [np.int32(2020), np.int32(2021), np.int32(2022)]


## Análise por Ano

### 2020

No ano de 2020, foram analisados um total de 320.751 voos, dos quais 35.921, correspondendo a 11,20%, foram classificados como atrasados. Entre os aeroportos estudados, o Aeroporto Internacional de Miami destacou-se por registrar o maior número de atrasos, totalizando 1.090 incidentes. Em seguida, surgem o Aeroporto de Frankfurt e o Aeroporto Schiphol de Amsterdã, que também apresentaram um número significativo de voos fora do horário previsto, demonstrando que certos hubs internacionais enfrentam desafios estruturais ou operacionais que impactam a pontualidade de suas operações aéreas.


In [140]:
# ---------------------------
# GRÁFICOS E RESUMOS - Ano 2020
# ---------------------------
ano = 2020
df_ano = df_realizados[df_realizados['Ano'] == ano].copy()
if df_ano.empty:
    print(f"Ano {ano}: sem registros válidos. Pulando.")
else:
    total_voos = len(df_ano)
    total_atrasados = int(df_ano['Voo Atrasado'].sum())
    perc_atrasados = (total_atrasados / total_voos * 100) if total_voos > 0 else 0.0

    # Top 10 aeroportos com mais atrasos
    atrasos_aeroporto = df_ano.groupby('name')['Voo Atrasado'].sum().reset_index().sort_values('Voo Atrasado', ascending=False)
    top10 = atrasos_aeroporto.head(10)
    fig1 = px.bar(top10[::-1], x='Voo Atrasado', y='name', orientation='h',
                  title=f'Top 10 Aeroportos com Mais Atrasos em {ano}',
                  labels={'name':'Aeroporto','Voo Atrasado':'Número de Atrasos'})
    fig1.show()
    print()

A análise das tendências mensais por aeroporto indicou que Miami International Airport apresentou a maior tendência de aumento de atrasos, com inclinação de 7,73 atrasos por mês, seguido por Toussaint Louverture International Airport (5,00), London Heathrow Airport (3,25), Frankfurt Airport (3,08) e Tocumen International Airport (3,04). Em contraste, aeroportos como Punta Cana International Airport (-10,00), Orlando International Airport (-4,88), Adolfo Suárez Madrid–Barajas Airport (-2,81), Rome–Fiumicino Leonardo da Vinci International Airport (-1,45) e John F Kennedy International Airport (-0,93) apresentaram tendência de redução de atrasos, indicando que algumas localidades conseguiram melhorar a pontualidade ao longo do ano.

In [141]:

    # Tendência por aeroporto (mensal)
    df_ano['Mes'] = df_ano['Partida Prevista'].dt.to_period('M').astype(str)
    df_ano['mes_num'] = df_ano['Partida Prevista'].dt.month
    agg_mensal = df_ano.groupby(['name','Mes','mes_num'])['Voo Atrasado'].sum().reset_index()

    slopes = []
    for aero, group in agg_mensal.groupby('name'):
        if len(group) >= 2:
            slope, _ = robust_linear_fit(group['mes_num'].values, group['Voo Atrasado'].values)
            slopes.append((aero, slope, int(group['Voo Atrasado'].sum())))
    slopes_df = pd.DataFrame(slopes, columns=['Aeroporto','Slope','Total Atrasos'])
    slopes_df = slopes_df[slopes_df['Total Atrasos'] >= 5]

    if not slopes_df.empty:
        top_increasing = slopes_df.sort_values('Slope', ascending=False).head(5)
        top_decreasing = slopes_df.sort_values('Slope', ascending=True).head(5)

        for aero in top_increasing['Aeroporto']:
            g = agg_mensal[agg_mensal['name'] == aero].sort_values('mes_num')
            fig2 = px.line(g, x='mes_num', y='Voo Atrasado', markers=True, title=f'Tendência de Atrasos Mensais: {aero} em {ano}',
                           labels={'mes_num':'Mês','Voo Atrasado':'Número de Atrasos'})
            fig2.show()

        txt_inc = "; ".join([f"{row['Aeroporto']} (slope={row['Slope']:.2f}, total={row['Total Atrasos']})"
                             for _, row in top_increasing.iterrows()]) if not top_increasing.empty else "Nenhum"
        txt_dec = "; ".join([f"{row['Aeroporto']} (slope={row['Slope']:.2f}, total={row['Total Atrasos']})"
                             for _, row in top_decreasing.iterrows()]) if not top_decreasing.empty else "Nenhum"

    print()



A tendência mensal de atrasos em 2020 mostrou crescimento constante, com um total de 35.921 incidentes e uma inclinação de 77,32 atrasos por mês, evidenciando que, ao longo do ano, a ocorrência de atrasos aumentou de forma linear, possivelmente influenciada por fatores sazonais e operacionais que afetaram a programação de voos em diversos aeroportos.

In [142]:

    # Tendência geral mensal
    monthly = df_ano.groupby('Mes').agg(mes_num=('mes_num','first'), n_atrasos=('Voo Atrasado','sum')).reset_index().sort_values('mes_num')
    fig3 = px.area(monthly, x='mes_num', y='n_atrasos', title=f'Tendência Mensal de Atrasos em {ano}',
                   labels={'mes_num':'Mês','n_atrasos':'Número de Atrasos'})
    fig3.show()

    if len(monthly) >= 2:
        slope_monthly, intercept_monthly = robust_linear_fit(monthly['mes_num'].values, monthly['n_atrasos'].values)
        if slope_monthly > 0:
            direcao = "aumentando"
        elif slope_monthly < 0:
            direcao = "diminuindo"
        else:
            direcao = "estável"

    print()


Quanto à distribuição dos atrasos por dia da semana, a quinta-feira (Thursday) registrou o maior número de incidentes, com 6.294 atrasos, representando 17,52% do total. Outros dias com números significativos incluem sexta-feira (Friday) com 5.842 atrasos (16,3%), quarta-feira (Wednesday) com 5.633 (15,7%) e segunda-feira (Monday) com 5.476 (15,2%), enquanto terça-feira (Tuesday), sábado (Saturday) e domingo (Sunday) apresentaram 4.918, 3.899 e 3.859 atrasos, respectivamente, demonstrando variações relevantes ao longo da semana.

In [143]:

    # Dias da semana com mais atrasos
    dias_semana = df_ano.groupby('Dia Semana')['Voo Atrasado'].sum().reset_index()
    fig4 = px.pie(dias_semana, names='Dia Semana', values='Voo Atrasado', title=f'Distribuição de Atrasos por Dia da Semana em {ano}')
    fig4.show()

    if not dias_semana.empty and dias_semana['Voo Atrasado'].sum() > 0:
        dias_semana['pct'] = dias_semana['Voo Atrasado'] / dias_semana['Voo Atrasado'].sum() * 100
        top_dia = dias_semana.sort_values('Voo Atrasado', ascending=False).iloc[0]

    print()


A análise do período do dia revelou que a tarde concentrou o maior número de atrasos, com 11.506 incidentes, correspondendo a 32,03% do total. A noite registrou 11.367 atrasos, a manhã 9.191 e a madrugada 3.857, indicando que os voos programados para períodos de maior movimentação enfrentaram maiores desafios operacionais e logísticos que impactaram a pontualidade.

In [144]:

    # Período do dia com mais atrasos
    periodo_dia = df_ano.groupby('Periodo Dia')['Voo Atrasado'].sum().reset_index()
    fig5 = px.bar(periodo_dia.sort_values('Voo Atrasado', ascending=False), x='Periodo Dia', y='Voo Atrasado',
                  title=f'Atrasos por Período do Dia em {ano}',
                  labels={'Periodo Dia':'Período do Dia','Voo Atrasado':'Número de Atrasos'})
    fig5.show()

    if not periodo_dia.empty and periodo_dia['Voo Atrasado'].sum() > 0:
        periodo_dia_sorted = periodo_dia.sort_values('Voo Atrasado', ascending=False)
        melhor_periodo = periodo_dia_sorted.iloc[0]
        pct_periodo = (melhor_periodo['Voo Atrasado'] / total_atrasados * 100) if total_atrasados > 0 else 0.0

    print()


/tmp/ipython-input-2512204784.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Em relação às companhias aéreas, as empresas com maior número de atrasos em 2020 foram AZU, com 11.122 incidentes, seguida por GLO com 9.940 e TAM com 4.360. Outras companhias relevantes incluíram OWT (948), LCO (920), LTG (899), PTB (601), GTI (500), SID (471) e PAM (340), mostrando que tanto o volume de operações quanto questões operacionais internas influenciaram a frequência de atrasos entre diferentes operadores.

In [145]:

    # Companhias aéreas com mais atrasos
    if coluna_companhia in df_ano.columns:
        companhia_atrasos = df_ano.groupby(coluna_companhia)['Voo Atrasado'].sum().reset_index().sort_values('Voo Atrasado', ascending=False)
        if not companhia_atrasos.empty:
            top_comp = companhia_atrasos.head(10)
            fig6 = px.treemap(top_comp, path=[coluna_companhia], values='Voo Atrasado',
                              title=f'Companhias com Mais Atrasos em {ano}')
            fig6.show()
    print()


### 2021

Em 2021, foram analisados 584.435 voos, dos quais 68.511, correspondendo a 11,72%, apresentaram atrasos superiores a 15 minutos na partida. Entre os aeroportos com maior número de atrasos, destacaram-se o Aeroporto Internacional de Miami, com 1.945 incidentes, seguido pelo Aeroporto Internacional de Tocumen, com 773, e pelo Aeroporto Humberto Delgado de Lisboa, com 659 atrasos. Outros aeroportos relevantes incluíram Amsterdam Schiphol, Frankfurt, Charles de Gaulle e Memphis, evidenciando que hubs internacionais de grande movimento enfrentam desafios operacionais significativos que impactam a pontualidade dos voos.


In [146]:
# ---------------------------
# GRÁFICOS E RESUMOS - Ano 2021
# ---------------------------
ano = 2021
df_ano = df_realizados[df_realizados['Ano'] == ano].copy()
if df_ano.empty:
    print(f"Ano {ano}: sem registros válidos. Pulando.")
else:
    total_voos = len(df_ano)
    total_atrasados = int(df_ano['Voo Atrasado'].sum())
    perc_atrasados = (total_atrasados / total_voos * 100) if total_voos > 0 else 0.0

    # Top 10 aeroportos com mais atrasos
    atrasos_aeroporto = df_ano.groupby('name')['Voo Atrasado'].sum().reset_index().sort_values('Voo Atrasado', ascending=False)
    top10 = atrasos_aeroporto.head(10)
    fig1 = px.bar(top10[::-1], x='Voo Atrasado', y='name', orientation='h',
                  title=f'Top 10 Aeroportos com Mais Atrasos em {ano}',
                  labels={'name':'Aeroporto','Voo Atrasado':'Número de Atrasos'})
    fig1.show()
    print()


Em termos de tendências por aeroporto ao longo do ano, observou-se que alguns locais apresentaram aumento consistente no número de atrasos. Tocumen International Airport registrou a maior inclinação de crescimento, com slope de 11,96, seguido por Lisboa, com slope de 11,61, e Miami, com slope de 8,18. Outros aeroportos, como Charles de Gaulle e Schiphol, também apresentaram crescimento moderado. Em contraste, aeroportos como Murtala Muhammed, Brussels, Memphis, Addis Ababa e Kotoka mostraram leve redução nos atrasos, com valores negativos de slope, indicando que, apesar do aumento global, algumas regiões conseguiram manter ou melhorar a pontualidade de suas operações.

In [147]:

    # Tendência por aeroporto (mensal)
    df_ano['Mes'] = df_ano['Partida Prevista'].dt.to_period('M').astype(str)
    df_ano['mes_num'] = df_ano['Partida Prevista'].dt.month
    agg_mensal = df_ano.groupby(['name','Mes','mes_num'])['Voo Atrasado'].sum().reset_index()

    slopes = []
    for aero, group in agg_mensal.groupby('name'):
        if len(group) >= 2:
            slope, _ = robust_linear_fit(group['mes_num'].values, group['Voo Atrasado'].values)
            slopes.append((aero, slope, int(group['Voo Atrasado'].sum())))
    slopes_df = pd.DataFrame(slopes, columns=['Aeroporto','Slope','Total Atrasos'])
    slopes_df = slopes_df[slopes_df['Total Atrasos'] >= 5]

    if not slopes_df.empty:
        top_increasing = slopes_df.sort_values('Slope', ascending=False).head(5)
        top_decreasing = slopes_df.sort_values('Slope', ascending=True).head(5)

        for aero in top_increasing['Aeroporto']:
            g = agg_mensal[agg_mensal['name'] == aero].sort_values('mes_num')
            fig2 = px.line(g, x='mes_num', y='Voo Atrasado', markers=True, title=f'Tendência de Atrasos Mensais: {aero} em {ano}',
                           labels={'mes_num':'Mês','Voo Atrasado':'Número de Atrasos'})
            fig2.show()

        txt_inc = "; ".join([f"{row['Aeroporto']} (slope={row['Slope']:.2f}, total={row['Total Atrasos']})"
                             for _, row in top_increasing.iterrows()]) if not top_increasing.empty else "Nenhum"
        txt_dec = "; ".join([f"{row['Aeroporto']} (slope={row['Slope']:.2f}, total={row['Total Atrasos']})"
                             for _, row in top_decreasing.iterrows()]) if not top_decreasing.empty else "Nenhum"

    print()



A análise da tendência mensal de atrasos mostrou um crescimento linear significativo ao longo de 2021, com uma inclinação de 822,74 atrasos por mês, refletindo um aumento contínuo do número de voos atrasados ao longo do ano. Esse comportamento sugere pressões operacionais crescentes, possivelmente relacionadas à recuperação do tráfego aéreo e ao aumento da demanda em determinados períodos, evidenciando a necessidade de estratégias de mitigação mais eficazes para reduzir atrasos.

In [148]:

    # Tendência geral mensal
    monthly = df_ano.groupby('Mes').agg(mes_num=('mes_num','first'), n_atrasos=('Voo Atrasado','sum')).reset_index().sort_values('mes_num')
    fig3 = px.area(monthly, x='mes_num', y='n_atrasos', title=f'Tendência Mensal de Atrasos em {ano}',
                   labels={'mes_num':'Mês','n_atrasos':'Número de Atrasos'})
    fig3.show()

    if len(monthly) >= 2:
        slope_monthly, intercept_monthly = robust_linear_fit(monthly['mes_num'].values, monthly['n_atrasos'].values)
        if slope_monthly > 0:
            direcao = "aumentando"
        elif slope_monthly < 0:
            direcao = "diminuindo"
        else:
            direcao = "estável"
    print()

Quando se observou a distribuição dos atrasos ao longo da semana, verificou-se que a sexta-feira foi o dia mais crítico, registrando 11.401 atrasos, representando 16,64% do total. Os demais dias também apresentaram números expressivos, com quinta-feira (11.011), segunda-feira (10.000), quarta-feira (9.901), terça-feira (9.552), domingo (8.426) e sábado (8.220), evidenciando padrões consistentes de maior movimento nos dias úteis e dias próximos ao final de semana, o que influencia diretamente a ocorrência de atrasos.

In [149]:

    # Dias da semana com mais atrasos
    dias_semana = df_ano.groupby('Dia Semana')['Voo Atrasado'].sum().reset_index()
    fig4 = px.pie(dias_semana, names='Dia Semana', values='Voo Atrasado', title=f'Distribuição de Atrasos por Dia da Semana em {ano}')
    fig4.show()

    if not dias_semana.empty and dias_semana['Voo Atrasado'].sum() > 0:
        dias_semana['pct'] = dias_semana['Voo Atrasado'] / dias_semana['Voo Atrasado'].sum() * 100
        top_dia = dias_semana.sort_values('Voo Atrasado', ascending=False).iloc[0]

    print()

Quanto à distribuição dos atrasos ao longo do dia, o período da tarde concentrou o maior número de incidentes, com 23.507 atrasos, equivalente a 34,31% do total. A noite registrou 20.866 atrasos, a manhã 17.262 e a madrugada 6.876, indicando que o acúmulo de voos nos horários de pico, principalmente à tarde, é um fator determinante para a ocorrência de atrasos e deve ser considerado no planejamento operacional e na gestão do tráfego aéreo.

In [150]:

    # Período do dia com mais atrasos
    periodo_dia = df_ano.groupby('Periodo Dia')['Voo Atrasado'].sum().reset_index()
    fig5 = px.bar(periodo_dia.sort_values('Voo Atrasado', ascending=False), x='Periodo Dia', y='Voo Atrasado',
                  title=f'Atrasos por Período do Dia em {ano}',
                  labels={'Periodo Dia':'Período do Dia','Voo Atrasado':'Número de Atrasos'})
    fig5.show()

    if not periodo_dia.empty and periodo_dia['Voo Atrasado'].sum() > 0:
        periodo_dia_sorted = periodo_dia.sort_values('Voo Atrasado', ascending=False)
        melhor_periodo = periodo_dia_sorted.iloc[0]
        pct_periodo = (melhor_periodo['Voo Atrasado'] / total_atrasados * 100) if total_atrasados > 0 else 0.0
    print()


/tmp/ipython-input-1683024770.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Em relação às companhias aéreas, aquelas com maior número de atrasos em 2021 foram AZU, com 20.012 incidentes, seguida por GLO, com 16.030, e TAM, com 11.265. Outras companhias relevantes incluíram CMP, LCO, LTG, SID, PAM, IPM e PTB, com atrasos variando entre 1.051 e 1.349 incidentes. Esses números indicam que a performance operacional das empresas também exerce influência significativa sobre a pontualidade dos voos, sendo essencial que cada companhia implemente medidas de eficiência e controle para reduzir atrasos.

In [151]:
    # Companhias aéreas com mais atrasos
    if coluna_companhia in df_ano.columns:
        companhia_atrasos = df_ano.groupby(coluna_companhia)['Voo Atrasado'].sum().reset_index().sort_values('Voo Atrasado', ascending=False)
        if not companhia_atrasos.empty:
            top_comp = companhia_atrasos.head(10)
            fig6 = px.treemap(top_comp, path=[coluna_companhia], values='Voo Atrasado',
                              title=f'Companhias com Mais Atrasos em {ano}')
            fig6.show()
    print()

### 2022

No ano de 2022, foram analisados 810.421 voos, dos quais 120.046, correspondendo a 14,81%, apresentaram atrasos superiores a 15 minutos na partida. Entre os aeroportos mais impactados, o Lisbon Humberto Delgado Airport liderou com 2.107 atrasos, seguido pelo Miami International Airport, com 1.971 incidentes, e pelo Tocumen International Airport, com 1.926. Outros aeroportos relevantes nesse ranking incluíram Amsterdam Airport Schiphol (834), Adolfo Suárez Madrid–Barajas Airport (680), Frankfurt Airport (581), Charles de Gaulle International Airport (575), John F. Kennedy International Airport (356), London Heathrow Airport (350) e George Bush Intercontinental Houston Airport (315), demonstrando que os atrasos se concentraram em hubs internacionais de grande movimentação.



In [152]:
# ---------------------------
# GRÁFICOS E RESUMOS - Ano 2021
# ---------------------------
ano = 2022
df_ano = df_realizados[df_realizados['Ano'] == ano].copy()
if df_ano.empty:
    print(f"Ano {ano}: sem registros válidos. Pulando.")
else:
    total_voos = len(df_ano)
    total_atrasados = int(df_ano['Voo Atrasado'].sum())
    perc_atrasados = (total_atrasados / total_voos * 100) if total_voos > 0 else 0.0


    # Top 10 aeroportos com mais atrasos
    atrasos_aeroporto = df_ano.groupby('name')['Voo Atrasado'].sum().reset_index().sort_values('Voo Atrasado', ascending=False)
    top10 = atrasos_aeroporto.head(10)
    fig1 = px.bar(top10[::-1], x='Voo Atrasado', y='name', orientation='h',
                  title=f'Top 10 Aeroportos com Mais Atrasos em {ano}',
                  labels={'name':'Aeroporto','Voo Atrasado':'Número de Atrasos'})
    fig1.show()
    print()


A análise das tendências mensais por aeroporto revelou que alguns aeroportos apresentaram aumento expressivo de atrasos ao longo do ano. Lisbon Humberto Delgado Airport destacou-se com inclinação de 22,33 atrasos por mês, seguido por Tocumen International Airport (5,56), Jomo Kenyatta International Airport (4,77), Charles de Gaulle International Airport (4,76) e Adolfo Suárez Madrid–Barajas Airport (2,29). Em contrapartida, aeroportos como Miami International Airport (-7,06), Amílcar Cabral International Airport (-1,10), Newark Liberty International Airport (-0,86), Los Angeles International Airport (-0,65) e Memphis International Airport (-0,59) apresentaram tendência de redução de atrasos, indicando variações regionais significativas nos padrões de pontualidade.

In [153]:

    # Tendência por aeroporto (mensal)
    df_ano['Mes'] = df_ano['Partida Prevista'].dt.to_period('M').astype(str)
    df_ano['mes_num'] = df_ano['Partida Prevista'].dt.month
    agg_mensal = df_ano.groupby(['name','Mes','mes_num'])['Voo Atrasado'].sum().reset_index()

    slopes = []
    for aero, group in agg_mensal.groupby('name'):
        if len(group) >= 2:
            slope, _ = robust_linear_fit(group['mes_num'].values, group['Voo Atrasado'].values)
            slopes.append((aero, slope, int(group['Voo Atrasado'].sum())))
    slopes_df = pd.DataFrame(slopes, columns=['Aeroporto','Slope','Total Atrasos'])
    slopes_df = slopes_df[slopes_df['Total Atrasos'] >= 5]

    if not slopes_df.empty:
        top_increasing = slopes_df.sort_values('Slope', ascending=False).head(5)
        top_decreasing = slopes_df.sort_values('Slope', ascending=True).head(5)

        for aero in top_increasing['Aeroporto']:
            g = agg_mensal[agg_mensal['name'] == aero].sort_values('mes_num')
            fig2 = px.line(g, x='mes_num', y='Voo Atrasado', markers=True, title=f'Tendência de Atrasos Mensais: {aero} em {ano}',
                           labels={'mes_num':'Mês','Voo Atrasado':'Número de Atrasos'})
            fig2.show()

        txt_inc = "; ".join([f"{row['Aeroporto']} (slope={row['Slope']:.2f}, total={row['Total Atrasos']})"
                             for _, row in top_increasing.iterrows()]) if not top_increasing.empty else "Nenhum"
        txt_dec = "; ".join([f"{row['Aeroporto']} (slope={row['Slope']:.2f}, total={row['Total Atrasos']})"
                             for _, row in top_decreasing.iterrows()]) if not top_decreasing.empty else "Nenhum"

    print()



A tendência linear de atrasos ao longo de 2022 apontou para um aumento consistente, com 120.046 incidentes registrados e uma inclinação de 654,41 atrasos por mês. Esse crescimento evidencia que os atrasos não foram distribuídos de forma uniforme ao longo do ano e que fatores operacionais, sazonais e logísticos contribuíram para o aumento progressivo de voos fora do horário programado.

In [154]:

    # Tendência geral mensal
    monthly = df_ano.groupby('Mes').agg(mes_num=('mes_num','first'), n_atrasos=('Voo Atrasado','sum')).reset_index().sort_values('mes_num')
    fig3 = px.area(monthly, x='mes_num', y='n_atrasos', title=f'Tendência Mensal de Atrasos em {ano}',
                   labels={'mes_num':'Mês','n_atrasos':'Número de Atrasos'})
    fig3.show()

    if len(monthly) >= 2:
        slope_monthly, intercept_monthly = robust_linear_fit(monthly['mes_num'].values, monthly['n_atrasos'].values)
        if slope_monthly > 0:
            direcao = "aumentando"
        elif slope_monthly < 0:
            direcao = "diminuindo"
        else:
            direcao = "estável"

    print()

Quando se observa a distribuição dos atrasos pelos dias da semana, a sexta-feira concentrou o maior número de incidentes, com 20.162 casos, correspondendo a 16,80% do total. Outros dias com alto número de atrasos incluíram terça-feira (17.364, 14,5%), quarta-feira (17.421, 14,5%) e segunda-feira (18.173, 15,1%). Dias de menor impacto foram sábado (13.324, 11,1%) e domingo (13.938, 11,6%), enquanto quinta-feira registrou 16.664 atrasos (16,4%), mostrando padrões semanais consistentes de maior congestão operacional em dias úteis.

In [155]:

    # Dias da semana com mais atrasos
    dias_semana = df_ano.groupby('Dia Semana')['Voo Atrasado'].sum().reset_index()
    fig4 = px.pie(dias_semana, names='Dia Semana', values='Voo Atrasado', title=f'Distribuição de Atrasos por Dia da Semana em {ano}')
    fig4.show()

    if not dias_semana.empty and dias_semana['Voo Atrasado'].sum() > 0:
        dias_semana['pct'] = dias_semana['Voo Atrasado'] / dias_semana['Voo Atrasado'].sum() * 100
        top_dia = dias_semana.sort_values('Voo Atrasado', ascending=False).iloc[0]

    print()

A análise por período do dia mostrou que a tarde foi o momento com maior incidência de atrasos, totalizando 42.357 incidentes, o que representa 35,28% do total. A manhã também apresentou número expressivo de atrasos, com 30.298 voos, seguida pela noite, com 20.866, e a madrugada, com 10.123 atrasos. Esses dados indicam que os períodos com maior concentração de voos diurnos tendem a apresentar maior probabilidade de atrasos, refletindo a pressão sobre a capacidade operacional e a logística aeroportuária.

In [156]:

    # Período do dia com mais atrasos
    periodo_dia = df_ano.groupby('Periodo Dia')['Voo Atrasado'].sum().reset_index()
    fig5 = px.bar(periodo_dia.sort_values('Voo Atrasado', ascending=False), x='Periodo Dia', y='Voo Atrasado',
                  title=f'Atrasos por Período do Dia em {ano}',
                  labels={'Periodo Dia':'Período do Dia','Voo Atrasado':'Número de Atrasos'})
    fig5.show()

    if not periodo_dia.empty and periodo_dia['Voo Atrasado'].sum() > 0:
        periodo_dia_sorted = periodo_dia.sort_values('Voo Atrasado', ascending=False)
        melhor_periodo = periodo_dia_sorted.iloc[0]
        pct_periodo = (melhor_periodo['Voo Atrasado'] / total_atrasados * 100) if total_atrasados > 0 else 0.0

    print()


/tmp/ipython-input-2512204784.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



No que se refere às companhias aéreas, as empresas com maior número de atrasos em 2022 foram GLO, com 32.220 incidentes, AZU, com 27.770, e TAM, com 25.651. Outras companhias também apresentaram números significativos, incluindo ACN (3.127), TAP (3.072), CMP (3.009), PTB (2.557), ARG (1.959), SID (1.452) e PAM (1.162). Esses dados evidenciam que a pontualidade das operações está fortemente ligada à gestão operacional de cada companhia, à programação de horários e ao volume de voos operados.

In [157]:
    # Companhias aéreas com mais atrasos
    if coluna_companhia in df_ano.columns:
        companhia_atrasos = df_ano.groupby(coluna_companhia)['Voo Atrasado'].sum().reset_index().sort_values('Voo Atrasado', ascending=False)
        if not companhia_atrasos.empty:
            top_comp = companhia_atrasos.head(10)
            fig6 = px.treemap(top_comp, path=[coluna_companhia], values='Voo Atrasado',
                              title=f'Companhias com Mais Atrasos em {ano}')
            fig6.show()
            resumo6 = f"Companhias com mais atrasos em {ano}: {top_list_to_text(companhia_atrasos, coluna_companhia, 'Voo Atrasado', top_n=10)}."
        else:
            resumo6 = f"Não há registros de companhia aérea no ano {ano}."
    else:
        resumo6 = f"Coluna '{coluna_companhia}' não encontrada no dataset para o ano {ano}."
    print()

# Conclusão:
Entre 2020 e 2022 observou-se aumento consistente na ocorrência de atrasos: a taxa global passou de 11,20% em 2020 para 14,81% em 2022, com crescimento linear expressivo (slope de 654,41 atrasos/mês em 2022). Houve mudança no epicentro dos impactos: em 2020 Miami figurava como principal fonte de atrasos; em 2022 Lisboa (Humberto Delgado) tornou-se o aeroporto com maior número absoluto de incidentes e a maior inclinação de aumento. Padrões temporais foram estáveis ao longo dos anos — maior concentração de atrasos em dias úteis finais (principalmente sexta-feira) e no período da tarde — e a responsabilidade pelos atrasos concentra-se em poucas companhias (AZU, GLO e TAM), com GLO liderando em 2022.


## Insights:

- Houve aumento sustentado de atrasos de 2020 → 2022; o volume absoluto e a inclinação mensal em 2022 indicam pressão crescente sobre capacidade e operação.

- Mudança geográfica de impacto: atenção operacional prioritária para Lisbon Humberto Delgado e Tocumen, que apresentam as maiores inclinações positivas.

- Alguns hubs reduziram atrasos (ex.: Miami com slope negativo em 2022), indicando que intervenções locais podem ser eficazes.

- Picos temporais claros — sexta-feira e período da tarde — sinalizam oportunidades para mitigações por gestão de slots, redistribuição de recursos e revisão de janelas operacionais.

- Intervenções por companhia aérea são necessárias: poucas operadoras concentram a maior parte dos atrasos, logo ações diretas com essas empresas apresentam alto potencial de impacto.

- Recomendações operacionais prioritárias (resumidas): fortalecer coordenação aeroporto–companhia, revisar alocação de slots e capacidade em horários de pico, implementar monitoramento preditivo das inclinações por aeroporto e executar auditorias operacionais direcionadas nos hubs com maior slope.